## Handle Image

In [6]:
import calendar

In [ ]:
from tkinter import *
from tkinter import filedialog
from tkinter import ttk
from PIL import Image, ImageTk
import os
import getpass as gt
from threading import Thread

class JupyterConfig:
    
    def __init__(self):
        
        # Start window
        self.root = Tk()
        self.root.config(background='white')
        self.root.title('Workspace Setup Configurator')
        self.root.resizable(width=False, height=False)
        self.root.lift()
        self.new_notebook_dir = "#c.NotebookApp.notebook_dir = ''\n"
        
        # Image Manipulation
        img = Image.open('jupyter.jpg')
        img = img.resize((150, 140))
        img = ImageTk.PhotoImage(master=self.root, image=img)
        image_label = ttk.Label(self.root, image=img, background='white')
        image_label.pack()    
        
        # Standard y paddings
        y_corner = 10
        y_sep = 1
        x_corner = 5
        x_sep = 1
        
        # LABELED FRAME STATUS
        # Starting the frame
        self.lframe = LabelFrame(self.root, text='Notebook Directory Status')
        self.lframe.config(background ='white')
        self.lframe.pack(fill='both', expand='yes', padx=10, pady=10)   
        
        # Load Jupyter Button
        self.load_button = ttk.Button(self.root, text='Launch Jupyter', command=self._jup_launch)
        self.load_button.config(state='normal')
        self.load_button.pack(side=BOTTOM, pady=(0, 20))
        
        s = ttk.Style()                     
        s.configure('Wild.TRadiobutton',
                background='white',         
                foreground='black')
        
        # Radio Selections -------------------------------
        var = IntVar()  
        var.set(1)
        self.destination = var      
        self.notebook = ttk.Radiobutton(self.root, text='Notebook', value=2, variable=var)
        self.notebook.config(style = 'Wild.TRadiobutton')
        self.notebook.pack(side=BOTTOM)
        self.lab = ttk.Radiobutton(self.root, text='Lab'+' '*11, value=1, variable=var)
        self.lab.config(style = 'Wild.TRadiobutton')
        self.lab.pack(side=BOTTOM)
        
        # Apply Button
        self.apply_button = ttk.Button(self.root, text='Apply New Directory', command=self._apply_directory)
        self.apply_button.config(state='disabled')
        self.apply_button.pack(side=BOTTOM, pady=(0, 20))
            
        # CANVAS 
        # Starting the canvas
        canvas = Canvas(self.root, background='white')
        canvas.pack(side=BOTTOM, padx=10, pady=10)
        
        # Grids in Canvas
        # Label
        config_label = ttk.Label(canvas, text='Jupyter Notebook Directory', background='white')
        config_label.grid(row=0, column=0, padx=(5, 10), pady=(y_corner, y_corner))       
        
        # Entry
        self.config_entry = ttk.Entry(canvas, width=60)
        self.config_entry.config(state='disabled')
        self.config_entry.grid(row=0, column=1, padx=(0, 0), pady=(y_corner, y_corner))
        
        # Button
        self.explorer_button = ttk.Button(canvas, text='explorer', command=self._file_explorer)
        self.explorer_button.config(state='disabled')
        self.explorer_button.grid(row=0, column=2, padx=(0, 10))
        
        self._populate_status()

        self.root.mainloop()
        
    def _get_notebook_dir(self):
        with open(self.config_file, 'r') as jupyter_config:
            doc = jupyter_config.readlines()
    
            for line in doc:
                if line.find('notebook_dir')>=0:
                    self.notebook_dir = line.lstrip(' ')
            jupyter_config.close()
        
    def _populate_status(self):
        # Standard y paddings
        y_corner = 10
        y_sep = 1
        x_corner = 5
        x_sep = 1
        
        # Get user configuration filename and path
        user = gt.getuser()
        self.config_file = rf"C:\Users\{user}\.jupyter\jupyter_notebook_config.py"
        
        if os.path.exists(self.config_file):  
            self.label_exist_desc = ttk.Label(self.lframe, text=f'Configuration File: {self.config_file}', 
                                              background='white')
            #self.label_exist_desc.grid(row=0, column=0, sticky='nw', padx=(x_corner, x_sep), pady=(y_corner, y_sep))
            self.label_exist_desc.pack(pady=(y_corner, y_sep))
            
            self._get_notebook_dir()
            
            self.label_dir_desc = ttk.Label(self.lframe, text=f'Notebook Directory: {self.notebook_dir}', background='white')
            #self.label_dir_desc.grid(row=1, column=0, sticky='nw', padx=(x_corner, x_sep), pady=(y_sep, y_corner))
            self.label_dir_desc.pack(pady=(y_sep, y_corner))
            
            self._activate()
        else:
            self.create_config = ttk.Button(self.root, text='Create New User Configuration', command=self._create_config)
            self.create_config.pack(side=TOP)
            
    def _repopulate_status(self):
        self._get_notebook_dir()
        self.label_dir_desc.config(text=f'Notebook Directory: {self.notebook_dir}')
        
    def _file_explorer(self):
        selection = filedialog.askdirectory(initialdir='')
        self.config_entry.delete(0, END)
        self.config_entry.insert(END, selection)
        
        replace_idx = self.notebook_dir.find('=')+1
        cut_notebook_dir = self.notebook_dir[:replace_idx].replace('#', '')
        
        self.new_notebook_dir = f"{cut_notebook_dir} '{selection}'\n"
        
        print(self.new_notebook_dir)
    
    def _create_config(self):
        os.system('jupyter notebook --generate-config')
        self.create_config.pack_forget()
        self._populate_status()
        
        
    def _activate(self):
        self.config_entry.config(state='normal')
        self.explorer_button.config(state='normal')
        self.apply_button.config(state='normal')
        
        self.root.update()
        
    def _apply_directory(self):
        with open(self.config_file, 'r') as jupyter_reconfig:
            doc = jupyter_reconfig.readlines()
            
            for i, line in enumerate(doc):
                if line.find('notebook_dir')>=0:
                    doc[i] = self.new_notebook_dir                    
            jupyter_reconfig.close()
        
        with open(self.config_file, 'w') as jupyter_reconfig:
            jupyter_reconfig.writelines(doc) 
            jupyter_reconfig.close()
            
        self._repopulate_status()
        self.root.update()
        
    def _jup_launch(self):
        print(self.destination.get())
        if self.destination.get() == 1:
            t = Thread(target = lambda: os.system("jupyter lab"))
        else:
            t = Thread(target = lambda: os.system("jupyter notebook"))
        t.start()        
                
JupyterConfig()

1
2
